In [2]:
# Please install OpenAI SDK first: `pip3 install openai`
# !pip install -U openai
import json
from openai import OpenAI
import pandas as pd
import torch
import datasets
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def generate_response(user_prompt, API_KEY):
    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
    system_prompt = """
    The user will provide some social media post text, parse and output in JSON format, "1" if the post is toxic, and "0" if not toxic. 
    
    EXAMPLE INPUT: 
    COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
    
    EXAMPLE JSON OUTPUT:
    {
        "cyberbullying": 1
    }
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={'type': 'json_object'},
        stream=False
    )
    return json.loads(response.choices[0].message.content)

def generate_response_multiclass(user_prompt, API_KEY):
    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
    system_prompt = """
    The user will provide some social media post text, parse and output in JSON format, multi-label classify the post for each of the following categories: toxic, severe toxic, obscene, threat, insult, identity hate. 
    
    EXAMPLE INPUT: 
    COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
    
    EXAMPLE JSON OUTPUT:
    {
        "toxic": 1,
        "severe toxic": 1,
        "obscene": 1,
        "threat": 0,
        "insult": 1,
        "identity hate": 0
    }
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={'type': 'json_object'},
        stream=False
    )
    return json.loads(response.choices[0].message.content)

def call_all(list_of_comments, dpsk_func, API_KEY):
    """Function to run the deepseek call on all the text data and output a JSON file"""
    json_outputs = []
    for comment in tqdm(list_of_comments, desc="Processing comments", unit="comment"):
        try:
            result = dpsk_func(comment, API_KEY)
            json_outputs.append(result)
        except Exception as e:
            print(f"Error on comment: {comment[:30]}... -> {e}")
            json_outputs.append({"error": str(e)})
    return json_outputs

Using cpu device


In [2]:
# Load the dataset
# df = pd.read_excel("train.xlsx")

# # Clean and prepare data
# df = df.dropna(subset=["comment_text", "cyberbullying"])
# df["comment_text"] = df["comment_text"].astype(str)
# df["cyberbullying"] = df["cyberbullying"].astype(int)

# def is_valid_text(t):
#     return isinstance(t, str) and len(t.strip()) > 0

# df = df[df["comment_text"].apply(is_valid_text)]
# comments = df["comment_text"].tolist()

# Load the dataset
dataset = datasets.load_from_disk("processed_dataset")
print(dataset)
train_data = dataset['train']
test_data = dataset['test']

DatasetDict({
    train: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cyberbullying'],
        num_rows: 159571
    })
    test: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cyberbullying'],
        num_rows: 63978
    })
})


In [8]:
# filter rows which are classified as cyberbullying
cyb_testdata = test_data.filter(lambda example: example['cyberbullying'] == 1)
# print(cyb_testdata[:5])
# filter rows which are classified as not cyberbullying
no_cyb_testdata = test_data.filter(lambda example: example['cyberbullying'] == 0)
# print(no_cyb_testdata[:5])

# randomly select 250 rows from cyb_testdata and 250 rows from no_cyb_testdata
cyb_testdata = cyb_testdata.shuffle(seed=100).select(range(250))
no_cyb_testdata = no_cyb_testdata.shuffle(seed=100).select(range(250))
# print(cyb_testdata[:5])
# print(no_cyb_testdata[:5])
assert cyb_testdata.features.type == no_cyb_testdata.features.type
sub_testdata = datasets.concatenate_datasets([cyb_testdata, no_cyb_testdata])
print(sub_testdata)

Dataset({
    features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cyberbullying'],
    num_rows: 500
})


In [9]:
# convert test_data to a list
test_list = sub_testdata['comment_text']
print(len(test_list[:]))

500


In [ ]:
API_KEY = "API KEY"
# binary label
json_outputs = call_all(test_list, generate_response, API_KEY)
json_array = json.dumps(json_outputs, indent=4)
with open('cyberbullying.json', 'w') as file:
    file.write(json_array)
    
# multi-label
json_outputs = call_all(test_list, generate_response_multiclass, API_KEY)
json_array = json.dumps(json_outputs, indent=4)
with open('cyberbullying_multi.json', 'w') as file:
    file.write(json_array)

# Evaluating the results

In [20]:
# first, join the json results with the existing data
sub_test = datasets.load_from_disk("subset_test")
print(sub_test)
cyb = datasets.load_dataset("json", data_files="cyberbullying_binary.json")['train']
cyb_multi = datasets.load_dataset("json", data_files="cyberbullying_multi.json")['train']
print(cyb)
print(cyb_multi)

Dataset({
    features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cyberbullying'],
    num_rows: 500
})
Dataset({
    features: ['cyberbullying'],
    num_rows: 500
})
Dataset({
    features: ['toxic', 'severe toxic', 'obscene', 'threat', 'insult', 'identity hate'],
    num_rows: 500
})


In [29]:
from sklearn.metrics import accuracy_score, f1_score, hamming_loss
import numpy as np
# Extract predictions and true labels
y_pred = cyb["cyberbullying"]
y_true = sub_test["cyberbullying"]

# Evaluate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Evaluate F1 score (micro or macro depending on your dataset)
f1 = f1_score(y_true, y_pred, average="macro")  # Use since classes are equally distributed in the dataset, micro or macro doesn't matter

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8460
F1 Score: 0.8459


In [32]:
# Define your target multi-label columns
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
pred_cols = ['toxic', 'severe toxic', 'obscene', 'threat', 'insult', 'identity hate']
# Map the references dataset to get multi-label vectors
sub_test = sub_test.map(lambda x: {'label': [x[col] for col in label_cols]})
# Map the predictions dataset similarly
cyb_multi = cyb_multi.map(lambda x: {'pred': [x[col] for col in pred_cols]})

# Extract predictions and true labels
y_pred = np.array(cyb_multi["pred"])
y_true = np.array(sub_test["label"])

# Compute metrics
subset_acc = accuracy_score(y_true, y_pred)
f1_micro = f1_score(y_true, y_pred, average="micro")
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_none = f1_score(y_true, y_pred, average=None)
# Hamming Loss, The Hamming loss is the fraction of labels that are incorrectly predicted.
hamming = hamming_loss(y_true, y_pred)

print(f"Subset Accuracy: {subset_acc:.4f}")
print(f"Hamming Loss: {hamming:.4f}")
print(f"F1 Score (Micro): {f1_micro:.4f}")
print(f"F1 Score (Macro): {f1_macro:.4f}")
print(f"F1 Score (each class): {f1_none}")

Map: 100%|██████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 7067.61 examples/s]


Subset Accuracy: 0.4940
Hamming Loss: 0.1403
F1 Score (Micro): 0.7029
F1 Score (Macro): 0.5761
F1 Score (each class): [0.85080645 0.28571429 0.75432526 0.34482759 0.70621469 0.51485149]
